In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
# for offline ploting
from plotly.offline import plot
import pandas as pd 
import math
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
from connect_mysql import query
plt.style.use('seaborn-poster')
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
PLOT_SIZE = (16, 10)

In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
updated_date = (datetime.datetime.today()-datetime.timedelta(days=2)).strftime('%m-%d-%Y')
latest_data = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{updated_date}.csv')
sum_country = query(keyw='Read', table_name='SUMMARY_COVID_DATA')[['NAME', 'CONTINENT', 'TOTAL_CASES', 'TOTAL_DEATHS', 'TOTAL_RECOVERED', 'ACTIVE_CASES', 'SERIOUS_CRITICAL', 'FATALITY_RATE']] ### query from database and get specified columns

In [23]:
sum_country

,NAME,CONTINENT,TOTAL_CASES,TOTAL_DEATHS,TOTAL_RECOVERED,ACTIVE_CASES,SERIOUS_CRITICAL,FATALITY_RATE
0,India,Asia,34322226,459685,33706533,156008,8944,1.34
1,Turkey,Asia,8121226,71298,7582624,467304,1405,0.88
2,Iran,Asia,5964824,126921,5560951,276952,3756,2.13
3,Indonesia,Asia,4246802,143500,4091938,11364,0,3.38
4,Philippines,Asia,2795642,43825,2714658,37159,2047,1.57
...,...,...,...,...,...,...,...,...
212,New Zealand,Australia/Oceania,6972,28,4759,2185,5,0.40
213,Wallis and Futuna,Australia/Oceania,445,7,438,0,0,1.57
214,Solomon Islands,Australia/Oceania,20,0,20,0,0,0.00
215,Vanuatu,Australia/Oceania,6,1,3,2,0,16.67


In [21]:
df_continents_cases = sum_country[['CONTINENT', 'TOTAL_CASES', 'TOTAL_DEATHS', 'TOTAL_RECOVERED', 'ACTIVE_CASES', 'SERIOUS_CRITICAL']]
df_continents_cases = df_continents_cases.groupby(by='CONTINENT').sum()
df_continents_cases

,TOTAL_CASES,TOTAL_DEATHS,TOTAL_RECOVERED,ACTIVE_CASES,SERIOUS_CRITICAL
CONTINENT,,,,,
Africa,8588145,219335,7953992,414818,2205
Asia,79723702,1176498,76804925,1742279,27489
Australia/Oceania,317839,3786,265178,48875,225
Europe,65381031,1316165,58652029,5412837,15247
North America,56559403,1153810,45331357,10074236,18689
South America,38466717,1171334,36887011,408372,12403


In [22]:
df_continents_cases["FATALITY_RATE (per 100)"] = np.round(100*df_continents_cases["TOTAL_DEATHS"]/df_continents_cases["TOTAL_CASES"],2)
df_continents_cases.style.background_gradient(cmap='Blues',subset=["TOTAL_CASES"])\
                        .background_gradient(cmap='Reds',subset=["TOTAL_DEATHS"])\
                        .background_gradient(cmap='Greens',subset=["TOTAL_RECOVERED"])\
                        .background_gradient(cmap='Purples',subset=["ACTIVE_CASES"])\
                        .background_gradient(cmap='Pastel1_r',subset=["SERIOUS_CRITICAL"])\
                        .background_gradient(cmap='YlOrBr',subset=["FATALITY_RATE (per 100)"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["TOTAL_CASES","TOTAL_DEATHS","TOTAL_RECOVERED","ACTIVE_CASES", 'SERIOUS_CRITICAL'])

,TOTAL_CASES,TOTAL_DEATHS,TOTAL_RECOVERED,ACTIVE_CASES,SERIOUS_CRITICAL,FATALITY_RATE (per 100)
CONTINENT,,,,,,
Africa,8588145,219335,7953992,414818,2205,2.55
Asia,79723702,1176498,76804925,1742279,27489,1.48
Australia/Oceania,317839,3786,265178,48875,225,1.19
Europe,65381031,1316165,58652029,5412837,15247,2.01
North America,56559403,1153810,45331357,10074236,18689,2.04
South America,38466717,1171334,36887011,408372,12403,3.05


In [34]:
### top 30 country with gradient
sum_country.drop(['CONTINENT'], axis=1).sort_values('TOTAL_CASES', ascending= False).head(30).set_index(['NAME']).style.background_gradient(cmap='Blues',subset=["TOTAL_CASES"])\
                        .background_gradient(cmap='Reds',subset=["TOTAL_DEATHS"])\
                        .background_gradient(cmap='Greens',subset=["TOTAL_RECOVERED"])\
                        .background_gradient(cmap='Purples',subset=["ACTIVE_CASES"])\
                        .background_gradient(cmap='Pastel1_r',subset=["SERIOUS_CRITICAL"])\
                        .background_gradient(cmap='YlOrBr',subset=["FATALITY_RATE"])\
                        .format("{:.2f}")\
                        .format("{:.0f}",subset=["TOTAL_CASES","TOTAL_DEATHS","TOTAL_RECOVERED","ACTIVE_CASES", 'SERIOUS_CRITICAL'])

,TOTAL_CASES,TOTAL_DEATHS,TOTAL_RECOVERED,ACTIVE_CASES,SERIOUS_CRITICAL,FATALITY_RATE
NAME,,,,,,
USA,47106268,770868,37116614,9218786,11875,1.64
India,34322226,459685,33706533,156008,8944,1.34
Brazil,21835785,608304,21035263,192218,8318,2.79
UK,9171660,141181,7462157,1568322,1030,1.54
Russia,8673860,243255,7477366,953239,2300,2.80
Turkey,8121226,71298,7582624,467304,1405,0.88
France,7180832,117802,6956474,106556,1049,1.64
Iran,5964824,126921,5560951,276952,3756,2.13
Argentina,5292549,116029,5158505,18015,596,2.19


In [31]:
sum_country.drop(['CONTINENT'], axis=1).sort_values('TOTAL_CASES', ascending= False).set_index(['NAME'])

,TOTAL_CASES,TOTAL_DEATHS,TOTAL_RECOVERED,ACTIVE_CASES,SERIOUS_CRITICAL,FATALITY_RATE
NAME,,,,,,
USA,47106268,770868,37116614,9218786,11875,1.64
India,34322226,459685,33706533,156008,8944,1.34
Brazil,21835785,608304,21035263,192218,8318,2.79
UK,9171660,141181,7462157,1568322,1030,1.54
Russia,8673860,243255,7477366,953239,2300,2.80
...,...,...,...,...,...,...
Vatican City,27,0,27,0,0,0.00
Solomon Islands,20,0,20,0,0,0.00
Western Sahara,10,1,8,1,0,10.00


In [ ]:
### reformat and generate csv file for running bar chart
def convert_df_running(origin_df, csv_name):
    df_running = origin_df.drop(['Lat', 'Long'], axis=1)
    df_running.columns = df_running.columns.str.replace('/', '-')
    df_running = df_running.groupby(by='Country-Region').sum()
    df_running.to_csv(rf'D:\PycharmProjects\Python\Covid-19\data\{csv_name}.csv')

In [ ]:
### generate csv file for running bar chart
convert_df_running(confirmed_df, 'confirmed_running')
convert_df_running(deaths_df, 'deaths_running')

In [ ]:
def plot_map(df, col):
    df = df[df[col]>0]
    fig = px.choropleth(df, locations="NAME", locationmode='country names', 
                  color=col, hover_name="NAME", 
                  title=col, hover_data=[col], color_continuous_scale='matter')                
    fig.show()

In [ ]:
plot_map(sum_country, 'TOTAL_CASES')
plot_map(sum_country, 'TOTAL_DEATHS')
plot_map(sum_country, 'TOTAL_RECOVERED')


In [ ]:
### scatter plot for top 25 country with total deaths and total cases 
fig = px.scatter(sum_country.sort_values('TOTAL_DEATHS', ascending=False).iloc[:25, :], 
                 x='TOTAL_CASES', y='TOTAL_DEATHS', color='NAME', size='TOTAL_CASES', 
                 height=700, text='NAME', log_x=True, log_y=True, 
                 title='Deaths vs Confirmed (Scale is in log10)')
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
### reformat for animation map
def convert_df_animation(origin_df, col):
    df_converted = origin_df.drop(['Lat', 'Long', 'Province/State'], axis=1) 
    df_converted = df_converted.set_index(['Country/Region']).stack().reset_index().rename(columns={'level_1':'Date', 0: col}) ### convert column dates to rows and rename
    df_converted["Date"] = df_converted["Date"].astype('datetime64[ns]')
    df_converted = df_converted.sort_values(by=['Date', 'Country/Region']) 
    return df_converted

In [ ]:
confirmed_df_converted = convert_df_animation(confirmed_df, 'Total_Confirmed')
deaths_df_converted = convert_df_animation(confirmed_df, 'Total_Deaths')

In [ ]:
### animation map for cases over time
fig = px.choropleth(confirmed_df_converted, locations="Country/Region", 
                    color=np.log(confirmed_df_converted["Total_Confirmed"]),
                    locationmode='country names', hover_name="Country/Region", 
                    animation_frame=confirmed_df_converted["Date"].dt.strftime('%Y-%m-%d'),
                    title='Cases over time', color_continuous_scale=px.colors.sequential.matter)
fig.update(layout_coloraxis_showscale=False)
plot(fig)

In [ ]:
### animation map for deaths over time
fig = px.choropleth(deaths_df_converted, locations="Country/Region", 
                    color=np.log(deaths_df_converted["Total_Deaths"]),
                    locationmode='country names', hover_name="Country/Region", 
                    animation_frame=deaths_df_converted["Date"].dt.strftime('%Y-%m-%d'),
                    title='Deaths over time', color_continuous_scale=px.colors.sequential.matter)
fig.update(layout_coloraxis_showscale=False)
plot(fig)

In [ ]:
cols = confirmed_df.keys()
### get data from 22/1/2021 to updated_date
confirmed = confirmed_df.loc[:, cols[4]:cols[-1]] 
deaths = deaths_df.loc[:, cols[4]:cols[-1]]

In [ ]:
dates = confirmed.keys()
world_cases = []
total_deaths = [] 
mortality_rate = []
for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    
    # confirmed, deaths
    world_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)

In [ ]:
def daily_increase(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d 

def moving_average(data, window_size):
    moving_average = []
    for i in range(len(data)):
        if i + window_size < len(data):
            moving_average.append(np.mean(data[i:i+window_size]))
        else:
            moving_average.append(np.mean(data[i:len(data)]))
    return moving_average

# window size
window = 7

# confirmed cases
world_daily_increase = daily_increase(world_cases)
world_confirmed_avg= moving_average(world_cases, window)
world_daily_increase_avg = moving_average(world_daily_increase, window)

# deaths
world_daily_death = daily_increase(total_deaths)
world_death_avg = moving_average(total_deaths, window)
world_daily_death_avg = moving_average(world_daily_death, window)

In [ ]:
num_of_days = np.array([i for i in range(len(dates))]) ### since 22/1/2020
world_cases = np.array(world_cases).reshape(-1, 1)
total_deaths = np.array(total_deaths).reshape(-1, 1)

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.plot(num_of_days, world_cases)
plt.plot(num_of_days, world_confirmed_avg, linestyle='dashed', color='red')
plt.title('Total of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Worldwide Coronavirus Cases', 'Moving Average {} Days'.format(window)], prop={'size': 15})
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.plot(num_of_days, total_deaths)
plt.plot(num_of_days, world_death_avg, linestyle='dashed', color='red')
plt.title('Total of Coronavirus Deaths Over Time', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Worldwide Coronavirus Deaths', 'Moving Average {} Days'.format(window)], prop={'size': 15})
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.bar(num_of_days, world_daily_increase)
plt.plot(num_of_days, world_daily_increase_avg, color='red', linestyle='dashed')
plt.title('World Daily Increases in Confirmed Cases', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Moving Average {} Days'.format(window), 'World Daily Increase in COVID-19 Cases'], prop={'size': 15})
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.bar(num_of_days, world_daily_death)
plt.plot(num_of_days, world_daily_death_avg, color='red', linestyle='dashed')
plt.title('World Daily Increases in Confirmed Deaths', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.legend(['Moving Average {} Days'.format(window), 'World Daily Increase in COVID-19 Deaths'], prop={'size': 15})
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.plot(num_of_days, np.log10(world_cases))
plt.title('Log of # of Coronavirus Cases Over Time', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
plt.figure(figsize=PLOT_SIZE)
plt.plot(num_of_days, np.log10(total_deaths))
plt.title('Log of # of Coronavirus Deaths Over Time', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('# of Cases', size=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
def country_plot(x, y1, y2, y3, country):
    # window is set as 14 in in the beginning of the notebook 
    confirmed_avg = moving_average(y1, window)
    confirmed_increase_avg = moving_average(y2, window)
    death_increase_avg = moving_average(y3, window)

    plt.figure(figsize=PLOT_SIZE)
    plt.plot(x, y1)
    plt.plot(x, confirmed_avg, color='red', linestyle='dashed')
    plt.legend(['{} Confirmed Cases'.format(country), 'Moving Average {} Days'.format(window)], prop={'size': 20})
    plt.title('{} Confirmed Cases'.format(country), size=30)
    plt.xlabel('Days Since 22/1/2020', size=30)
    plt.ylabel('# of Cases', size=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

    plt.figure(figsize=PLOT_SIZE)
    plt.bar(x, y2)
    plt.plot(x, confirmed_increase_avg, color='red', linestyle='dashed')
    plt.legend(['Moving Average {} Days'.format(window), '{} Daily Increase in Confirmed Cases'.format(country)], prop={'size': 20})
    plt.title('{} Daily Increases in Confirmed Cases'.format(country), size=30)
    plt.xlabel('Days Since 22/1/2020', size=30)
    plt.ylabel('# of Cases', size=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

    plt.figure(figsize=PLOT_SIZE)
    plt.bar(x, y3)
    plt.plot(x, death_increase_avg, color='red', linestyle='dashed')
    plt.legend(['Moving Average {} Days'.format(window), '{} Daily Increase in Confirmed Deaths'.format(country)], prop={'size': 20})
    plt.title('{} Daily Increases in Deaths'.format(country), size=30)
    plt.xlabel('Days Since 22/1/2020', size=30)
    plt.ylabel('# of Cases', size=30)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

In [ ]:
# helper function for getting country's cases, deaths, and recoveries        
def get_country_info(country_name):
    country_cases = []
    country_deaths = []
    for i in dates:
        country_cases.append(confirmed_df[confirmed_df['Country/Region']==country_name][i].sum())
        country_deaths.append(deaths_df[deaths_df['Country/Region']==country_name][i].sum())
    return (country_cases, country_deaths)

In [ ]:
def country_visualizations(country_name):
    country_info = get_country_info(country_name)
    country_cases = country_info[0]
    country_deaths = country_info[1]
    
    country_daily_increase = daily_increase(country_cases)
    country_daily_death = daily_increase(country_deaths)
    country_plot(num_of_days, country_cases, country_daily_increase, country_daily_death, country_name)

In [ ]:
country = input('Which country do you want to see:')
country_visualizations(country)

In [ ]:
unique_countries =  list(latest_data['Country_Region'].unique())
country_confirmed_cases = []
country_death_cases = [] 
country_active_cases = []
country_incidence_rate = [] 
country_mortality_rate = [] 
no_cases = []
for i in unique_countries:
    cases = latest_data[latest_data['Country_Region']==i]['Confirmed'].sum()
    if cases > 0:
        country_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
        
for i in no_cases:
    unique_countries.remove(i)
    
# sort countries by the number of confirmed cases
unique_countries = [k for k, v in sorted(zip(unique_countries, country_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
for i in range(len(unique_countries)):
    country_confirmed_cases[i] = latest_data[latest_data['Country_Region']==unique_countries[i]]['Confirmed'].sum()
    country_death_cases.append(latest_data[latest_data['Country_Region']==unique_countries[i]]['Deaths'].sum())
    country_incidence_rate.append(latest_data[latest_data['Country_Region']==unique_countries[i]]['Incident_Rate'].sum())
    country_mortality_rate.append(country_death_cases[i]/country_confirmed_cases[i])


In [ ]:
country_df = pd.DataFrame({'Country Name': unique_countries, 'Number of Confirmed Cases': [format(int(i), ',d') for i in country_confirmed_cases],
                          'Number of Deaths': [format(int(i), ',d') for i in country_death_cases], 
                          'Incidence Rate' : country_incidence_rate,
                          'Mortality Rate': country_mortality_rate})
                          
# number of cases per country/region
country_df.style.background_gradient(cmap='autumn_r')

In [ ]:
mean_mortality_rate = np.mean(mortality_rate)
plt.figure(figsize=(16, 10))
plt.plot(num_of_days, mortality_rate, color='red')
plt.axhline(y = mean_mortality_rate, linestyle='--', color='black')
plt.title('Worldwide Mortality Rate of Coronavirus Over Time', size=30)
plt.xlabel('Days Since 22/1/2020', size=30)
plt.ylabel('Case Mortality Rate', size=30)
plt.legend(['Mean_mortality_rate', 'Worldwide Mortality Rate'])
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
unique_provinces =  list(latest_data['Province_State'].unique())
province_confirmed_cases = []
province_country = [] 
province_death_cases = [] 
province_incidence_rate = []
province_mortality_rate = [] 

no_cases = [] 
for i in unique_provinces:
    cases = latest_data[latest_data['Province_State']==i]['Confirmed'].sum()
    if cases > 0:
        province_confirmed_cases.append(cases)
    else:
        no_cases.append(i)
 
# remove areas with no confirmed cases
for i in no_cases:
    unique_provinces.remove(i)
    
unique_provinces = [k for k, v in sorted(zip(unique_provinces, province_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
for i in range(len(unique_provinces)):
    province_confirmed_cases[i] = latest_data[latest_data['Province_State']==unique_provinces[i]]['Confirmed'].sum()
    province_country.append(latest_data[latest_data['Province_State']==unique_provinces[i]]['Country_Region'].unique()[0])
    province_death_cases.append(latest_data[latest_data['Province_State']==unique_provinces[i]]['Deaths'].sum())
    province_incidence_rate.append(latest_data[latest_data['Province_State']==unique_provinces[i]]['Incident_Rate'].sum())
    province_mortality_rate.append(province_death_cases[i]/province_confirmed_cases[i])

In [ ]:
# number of cases per province/state/city top 100 
province_limit = 100 
province_df = pd.DataFrame({'Province/State Name': unique_provinces[:province_limit], 'Country': province_country[:province_limit], 'Number of Confirmed Cases': [format(int(i), ',d') for i in province_confirmed_cases[:province_limit]],
                          'Number of Deaths': [format(int(i), ',d') for i in province_death_cases[:province_limit]], 
                        'Incidence Rate' : province_incidence_rate[:province_limit], 'Mortality Rate': province_mortality_rate[:province_limit]})
# number of cases per country/region

province_df.style.background_gradient(cmap='Oranges')

In [ ]:
# return the data table with province/state info for a given country
def country_table(country_name):
    states = list(latest_data[latest_data['Country_Region']==country_name]['Province_State'].unique())
    state_confirmed_cases = []
    state_death_cases = [] 
    state_incidence_rate = [] 
    state_mortality_rate = [] 

    no_cases = [] 
    for i in states:
        cases = latest_data[latest_data['Province_State']==i]['Confirmed'].sum()
        if cases > 0:
            state_confirmed_cases.append(cases)
        else:
            no_cases.append(i)

    # remove areas with no confirmed cases
    for i in no_cases:
        states.remove(i)

    states = [k for k, v in sorted(zip(states, state_confirmed_cases), key=operator.itemgetter(1), reverse=True)]
    for i in range(len(states)):
        state_confirmed_cases[i] = latest_data[latest_data['Province_State']==states[i]]['Confirmed'].sum()
        state_death_cases.append(latest_data[latest_data['Province_State']==states[i]]['Deaths'].sum())
        state_incidence_rate.append(latest_data[latest_data['Province_State']==states[i]]['Incident_Rate'].sum())
        state_mortality_rate.append(state_death_cases[i]/state_confirmed_cases[i])
        
      
    state_df = pd.DataFrame({'State Name': states, 'Number of Confirmed Cases': [format(int(i), ',d') for i in state_confirmed_cases],
                              'Number of Deaths': [format(int(i), ',d') for i in state_death_cases], 
                             'Incidence Rate' : state_incidence_rate, 'Mortality Rate': state_mortality_rate})
    # number of cases per country/region
    return state_df

In [ ]:
country_name = input('Input your country: ')
table = country_table(country_name)
table.style.background_gradient(cmap='Oranges')

In [ ]:
total_world_cases = np.sum(country_confirmed_cases)
us_confirmed = latest_data[latest_data['Country_Region']=='US']['Confirmed'].sum()
outside_us_confirmed = total_world_cases - us_confirmed

plt.figure(figsize=(16, 9))
plt.barh('United States', us_confirmed)
plt.barh('Outside United States', outside_us_confirmed)
plt.title('# of Total Coronavirus Confirmed Cases', size=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

plt.figure(figsize=(16, 9))
plt.barh('United States', us_confirmed/total_world_cases)
plt.barh('Outside United States', outside_us_confirmed/total_world_cases)
plt.title('# of Coronavirus Confirmed Cases Expressed in Percentage', size=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

In [ ]:
print('Outside United States {} cases ({}%):'.format(outside_us_confirmed, np.round((outside_us_confirmed/total_world_cases)*100, 1)))
print('United States {} cases ({}%)'.format(us_confirmed, np.round((us_confirmed/total_world_cases)*100, 1)))
print('Total: {} cases'.format(total_world_cases))

In [ ]:
# Only show 10 countries with the most confirmed cases, the rest are grouped into the other category
visual_unique_countries = [] 
visual_confirmed_cases = []
others = np.sum(country_confirmed_cases[10:])

for i in range(len(country_confirmed_cases[:10])):
    visual_unique_countries.append(unique_countries[i])
    visual_confirmed_cases.append(country_confirmed_cases[i])
    
visual_unique_countries.append('Others')
visual_confirmed_cases.append(others)

In [ ]:
def plot_bar_graphs(x, y, title):
    plt.figure(figsize=PLOT_SIZE)
    plt.barh(x, y)
    plt.title(title, size=20)
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

In [ ]:
plot_bar_graphs(visual_unique_countries, visual_confirmed_cases, '# of Covid-19 Confirmed Cases in Countries/Regions')

In [ ]:
log_country_confirmed_cases = [math.log10(i) for i in visual_confirmed_cases]
plot_bar_graphs(visual_unique_countries, log_country_confirmed_cases, 'Common Log # of Coronavirus Confirmed Cases in Countries/Regions')

In [ ]:
# Only show 10 provinces with the most confirmed cases, the rest are grouped into the other category
visual_unique_provinces = [] 
visual_confirmed_cases2 = []
others = np.sum(province_confirmed_cases[10:])
for i in range(len(province_confirmed_cases[:10])):
    visual_unique_provinces.append(unique_provinces[i])
    visual_confirmed_cases2.append(province_confirmed_cases[i])

visual_unique_provinces.append('Others')
visual_confirmed_cases2.append(others)

In [ ]:
plot_bar_graphs(visual_unique_provinces, visual_confirmed_cases2, '# of Coronavirus Confirmed Cases in Provinces/States')

In [ ]:
log_province_confirmed_cases = [math.log10(i) for i in visual_confirmed_cases2]
plot_bar_graphs(visual_unique_provinces, log_province_confirmed_cases, 'Log of # of Coronavirus Confirmed Cases in Provinces/States')

In [ ]:
def plot_pie_charts(x, y, title):
    # more muted color 
    c = ['lightcoral', 'rosybrown', 'sandybrown', 'navajowhite', 'gold',
        'khaki', 'lightskyblue', 'turquoise', 'lightslategrey', 'thistle', 'pink']
    plt.figure(figsize=(20,15))
    plt.title(title, size=20)
    plt.pie(y, colors=c,shadow=True)
    plt.legend(x, loc='best', fontsize=12)
    plt.show()

In [ ]:
plot_pie_charts(visual_unique_countries, visual_confirmed_cases, 'Covid-19 Confirmed Cases per Country')

In [ ]:
plot_pie_charts(visual_unique_provinces, visual_confirmed_cases2, 'Covid-19 Confirmed Cases per State/Province/Region')

In [ ]:
# Plotting countries with regional data using a pie chart 
def plot_pie_country_with_regions(country_name, title):
    regions = list(latest_data[latest_data['Country_Region']==country_name]['Province_State'].unique())
    confirmed_cases = []
    no_cases = [] 
    for i in regions:
        cases = latest_data[latest_data['Province_State']==i]['Confirmed'].sum()
        if cases > 0:
            confirmed_cases.append(cases)
        else:
            no_cases.append(i)

    # remove areas with no confirmed cases
    for i in no_cases:
        regions.remove(i)

    # only show the top 5 states
    regions = [k for k, v in sorted(zip(regions, confirmed_cases), key=operator.itemgetter(1), reverse=True)]

    for i in range(len(regions)):
        confirmed_cases[i] = latest_data[latest_data['Province_State']==regions[i]]['Confirmed'].sum()  
    
    # additional province/state will be considered "others"    
    if(len(regions)>5):
        regions_5 = regions[:5]
        regions_5.append('Others')
        confirmed_cases_5 = confirmed_cases[:5]
        confirmed_cases_5.append(np.sum(confirmed_cases[5:]))
        plot_pie_charts(regions_5,confirmed_cases_5, title)
    else:
        plot_pie_charts(regions,confirmed_cases, title)

In [ ]:
country_name = input('Input the country name: ')
plot_pie_country_with_regions(country_name, 'Covid-19 Confirmed Cases in {}'.format(i))

In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

In [ ]:
days_to_skip = 376
days_since_1_22 = np.array([i for i in range(len(dates))]).reshape(-1, 1)
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(days_since_1_22[days_to_skip:], world_cases[days_to_skip:], test_size=0.08, shuffle=False) 

In [ ]:
svm_confirmed = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed.fit(X_train_confirmed, y_train_confirmed)
svm_pred = svm_confirmed.predict(future_forcast)

In [ ]:
len(svm_pred)